# 0. Import/Classes/Logger

In [55]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException, ElementClickInterceptedException, InvalidSelectorException, WebDriverException

import re
from time import sleep as time_sleep, time
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED
from selenium import webdriver
import requests
from webdriver_manager.chrome import ChromeDriverManager
import lxml
from itertools import chain
from typing import NamedTuple
import pickle
from collections import deque
from tqdm import tqdm
import pandas as pd
import numpy as np
import logging
from pathlib import Path

In [3]:
# получение пользовательского логгера и установка уровня логирования
short_logger = logging.getLogger('short_logger')
short_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
short_handler = logging.FileHandler("data/short_logger.log", encoding='utf-8', mode='a')
short_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
short_handler.setFormatter(short_formatter)
# добавление обработчика к логгеру
short_logger.addHandler(short_handler)

# получение пользовательского логгера и установка уровня логирования
long_logger = logging.getLogger('long_logger')
long_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
long_handler = logging.FileHandler("data/long_logger.log", encoding='utf-8', mode='w')
long_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
long_handler.setFormatter(long_formatter)
# добавление обработчика к логгеру
long_logger.addHandler(long_handler)

In [4]:
def _save_df_to_zip(df_: pd.DataFrame, archive_name: str = 'archive', folder: str='data', replace: bool=False) -> None:
    # Путь к файлу
    file_path = Path(folder).joinpath(archive_name + '.zip')
    Path(folder).mkdir(exist_ok=True)
    # Проверяем, существует ли файл
    if file_path.exists() and not replace:
        # Получаем время создания файла
        time = datetime.fromtimestamp(file_path.lstat().st_atime).strftime('%Y-%m-%d %H:%M')

        # Создаем новое имя файла с добавлением времени Unix
        new_file_name = file_path.stem + "_" + str(time) + file_path.suffix

        # Создаем новый путь для переименованного файла
        new_file_path = file_path.with_name(new_file_name)
        # Переименовываем файл
        file_path.rename(new_file_path)

# to csv
    compression_opts = dict(method='zip', archive_name=f'{archive_name}.csv')
    df_.to_csv(f'{folder}/{archive_name}.zip', index=False, compression=compression_opts, encoding='utf-8')

In [5]:
class market_link(NamedTuple):
    sity: str
    title: str
    discounts: int
    alcohol: bool
    href: str
    su1: str
    su2: str
    page_href: str

class MyTimeoutErrore(Exception):
    def __init__(self, text: str='timeout'):
        self.txt = text

In [56]:
driver = webdriver.Chrome()

# 1. Получение адресов городов

# 2. Получение ссылок на Магазины

## 2.1. Получение ссылки на старицы каталогов с магазинами

### 2.1.1. Функции

In [36]:
def _get_page_count(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        # wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            print(ex)
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            print(ex)
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise Exception("timeout")
    
    return out_list

### 2.1.2. Выполнение

In [34]:
# driver.get('https://edadeal.ru/lipeck/retailers')
url = 'https://edadeal.ru/lipeck/retailers'

In [37]:
temp_url_list = _get_page_count(driver, url, '?page=')
temp_url_list

list index out of range


['https://edadeal.ru/lipeck/retailers?page=1',
 'https://edadeal.ru/lipeck/retailers?page=2',
 'https://edadeal.ru/lipeck/retailers?page=3',
 'https://edadeal.ru/lipeck/retailers?page=4',
 'https://edadeal.ru/lipeck/retailers?page=5']

## 2.2. Получение непосредственно все ссылки на Магазины

### 2.2.1. Функции

In [38]:
def get_retailers_links(driver: webdriver.Chrome, url: str, sity: str, timeout: int=30) -> list[market_link]:
    """
        Возвращает лист с магазинами и ссылками.
    """
    
    host = '/'.join(url.split('/')[:-1])
    out_list = list()
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1
        )

    
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('div', class_='p-dsk-srch-retailers__content').find('div', class_='b-dsk-grid__container').find_all('article')
        except Exception as ex:
            print(ex)
            continue
        
        try:    
            for item in f1:
                href = host + item.find('a').get('href')
                title = item.find('a').find('h4').get('title')
                discounts = int((re.findall(r'(\d+)', item.find('a').find('p').text)[0]))
                out_list.append(market_link(sity, title, discounts, False, href, None, None, None))
                
        except Exception as ex:
            print(ex)
            continue
        
        break
    
    else:
        raise MyTimeoutErrore('timeout')
    
    return out_list

### 2.2.2. Выполнение

In [39]:
temp_retailer_links = list()
for link in temp_url_list:
    temp_retailer_links.append(get_retailers_links(driver, link, 'lipetsk'))

retailer_links = list(chain.from_iterable(temp_retailer_links))
print(len(retailer_links))
# print(retailer_links)

59


### 2.2.3. Save/Load

In [40]:
with open('data/retailer_links.pickle', 'wb') as file:
    pickle.dump(retailer_links, file)

In [57]:
with open('data/retailer_links.pickle', 'rb') as file:
    retailer_links = pickle.load(file)

# 3. Получение адресов Товаров

## 3.1. Получение ссылок на каталоги с товарами

### 3.1.1. Функции

In [7]:
def _get_page_count2(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> tuple[list[str],bool]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    try:
        driver.get(url)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    
    empty_page = False
    # Клик по кнопке "Есть 18 лет"
    alcohol = True
    # Проверка появления "Да, мне есть 18"
    try:
        button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
        button.click()
    except TimeoutException:
        alcohol = False
    except ElementNotInteractableException:
        alcohol = False
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        except TimeoutException:
            try:
                # Проверка на отсутствие скидок.
                driver.find_element(by=By.XPATH, value="//div[@class='b-dsk-stub__description']")
            except TimeoutException:
                raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
            else:
                empty_page = True
                out_list = None
                alcohol = True
                break
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not PAGINATOR in url: {url}')
    
    return (out_list, alcohol, empty_page)

def _get_all_links_to_market(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=30) -> list[market_link]:
    """
        Получение адресов на все страницы с каталогами товаров
        Что бы после можно было использовать много поточность.
    """
    loger_fault = list()
    dead_line = 100
    # Очередь
    q = deque(list_url)

    out_list = list()
    with tqdm(total=len(list_url)) as pbar:
        while len(q) > 0:
            if dead_line < 0:
                break
            
            item = q.popleft()
            
            if item.discounts > 24:
                try:
                    temp_url_list, alcohol, empty_page = _get_page_count2(driver, item.href, '?page=', timeout)
                except MyTimeoutErrore as mte:
                    q.append(item)
                    loger_fault.append(mte)
                    dead_line -= 1
                    continue
                if not empty_page:
                    for href in temp_url_list:
                        out_list.append(market_link(item.sity,
                                                    item.title,
                                                    item.discounts,
                                                    alcohol,
                                                    item.href,
                                                    href))
                    
            elif item.discounts:
                out_list.append(market_link(item.sity,
                                            item.title,
                                            item.discounts,
                                            True,
                                            item.href,
                                            item.href))
            else:
                out_list.append(market_link(*item[:-1], None))
            
            pbar.update(1)

    return (out_list, loger_fault, q)

### 2.2.2/2 Корректировка

In [58]:
url = retailer_links[2].href
driver.get(url=url)
market = retailer_links[2]

In [59]:
def _check_load_page(driver: webdriver.Chrome, timeout: int=20) -> bool:
    """
       Проверка наличия названия продукта в ячейке
    """
    time_start = int(time())
    while (int(time()) - time_start) < 20:
        n = driver.find_elements(By.XPATH, """//div[contains(@class, "p-dsk-srch-retailer__content")]
                                                //div[contains(@class, "b-dsk-grid__container")]
                                                /a[contains(@class, "p-dsk-srch-retailer__card")]
                                                //div[contains(@class, "b-srch-card__price-new")]
                                                //span[@data-test-ref="money-base"]
                                                """)
            
        if len(n) > 0:
            if int(n[0].text.replace('\xa0', '').replace(' ', '')) > 0:
                return(True)
        else:
            continue
            time_sleep(0.2)
    return(False)



def _check_alcohol(driver: webdriver.Chrome, item: market_link, timeout: int=20) -> bool:
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1
        )

    try:
        # Ожидаем появляение кнопки, если она не повится, то ничего не делаем.
        print('alcohol')
        button = driver.find_element(By.XPATH, "//div[contains(text(), 'Да. Мне есть 18')]")
        button.click()
        return True
    except TimeoutException:
        short_logger.info(f'alcochol timeout: {item.title} :: {item.page_href}')
    except ElementNotInteractableException:
        short_logger.warning(f'alcochol not button: {item.title} :: {item.page_href}')
    except Exception as ex:
        short_logger.error(f'alcochol other: {ex} :: {item.title} :: {item.page_href}')
        long_logger.error(f'alcochol other: {ex} :: {item.title} :: {item.page_href}', exc_info=True)
    finally:
        return False



                
                
def get_link_to_pages_without_except(driver: webdriver.Chrome, timeout: int=20):
    out_list = list()
    
    def _get_link_to_pages(driver: webdriver.Chrome, timeout: int=20):
        wait = WebDriverWait(
            driver,
            timeout=timeout,
            poll_frequency=1
            )
                
        
        nonlocal out_list
        
        el1 = (driver
              .find_element(By.CSS_SELECTOR, '.p-dsk-srch-retailer__body')
              .find_element(By.CSS_SELECTOR, '.p-dsk-srch-retailer__sidebar-container')
              .find_element(By.CSS_SELECTOR, '.i-block-helper')
              .find_element(By.CSS_SELECTOR, '.b-dsk-srch-cats-tree__node')
              .find_elements(By.XPATH, "following-sibling::*")
              )
        
        for _ in range(10):
            try:
                driver.find_element(By.XPATH, '//div[contains(text(), "Да. Мне есть 18")]/ancestor::button').click()
            except NoSuchElementException:
                break
            except ElementNotInteractableException:
                time_sleep(0.2)
                continue
            else:
                break
        
        
        for el in el1:
                time_sleep(0.2)
                el2 = (el
                       .find_element(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__item-wrapper")]/button')
                      )
                el2.click()
                time_sleep(0.2)
                su1 = el2.text
                
                        
                if (_check_load_page(driver)):
                    # print(su1)
                    el3 = (el
                        .find_element(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__children")]/div[contains(@class, "i-block-helper")]')
                        .find_elements(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__node")]//button')
                        )
                else:
                    raise MyTimeoutErrore()
        
                for item in el3:
                    item.click()
                    time_sleep(0.2)
                    item.send_keys(Keys.ARROW_DOWN)
                    # item.send_keys(Keys.ARROW_DOWN)
                    if (_check_load_page(driver)):
                        su2 = item.text
        
                        su3_list = wait.until(lambda d: d.find_elements(By.XPATH, '//li[contains(@class, "b-dsk-srch-cats-list__item")]/button'))
                        # su3_list = driver.find_elements(By.XPATH, '//li[contains(@class, "b-dsk-srch-cats-list__item")]/button')
        
                        if len(su3_list) > 2:
                            for s_item in su3_list[1:]:
                                su3 = s_item.text
                                href = driver.current_url + '&segmentUuid2='+ s_item.get_attribute('value')
                                out_list.append((su1, su2, su3, href))
                        else:
                            su3 = np.nan
                            href = driver.current_url
                            out_list.append((su1, su2, su3, href))
    
    try:
        _get_link_to_pages(driver, timeout)
    except WebDriverException:
        pass
    print(out_list)

# out_html = el3[0]
# print(BS(out_html.get_attribute('innerHTML')).prettify())
# WebDriverException                        

In [60]:
get_link_to_pages_without_except(driver)

[('Продукты', 'Молочные продукты', 'Йогурт', 'https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b31b1a8-6311-11e6-849f-52540010b608&segmentUuid2=3b31b862-6311-11e6-849f-52540010b608'), ('Продукты', 'Молочные продукты', 'Сыр', 'https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b31b1a8-6311-11e6-849f-52540010b608&segmentUuid2=3b31b30e-6311-11e6-849f-52540010b608'), ('Продукты', 'Молочные продукты', 'Молоко', 'https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b31b1a8-6311-11e6-849f-52540010b608&segmentUuid2=3b31b275-6311-11e6-849f-52540010b608'), ('Продукты', 'Молочные продукты', 'Масло, маргарин, спред', 'https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b31b1a8-6311-11e6-849f-52540010b608&segmentUuid2=3b31b415-6311-11e6-849f-52540010b608'), ('Продукты', 'Молочные продукты', 'Сыр плавленый', 'https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b31b1a8-6311-11e6-849f-52540010b608&segmentUuid2=3b31b396-6311-11e6-849f-52540010b6

In [39]:
"""
   Проверка наличия названия продукта в ячейке
"""
time_start = int(time())
while (int(time()) - time_start) < 20:
    n = driver.find_elements(By.XPATH, """//div[contains(@class, "p-dsk-srch-retailer__content")]
                                            //div[contains(@class, "b-dsk-grid__container")]
                                            /a[contains(@class, "p-dsk-srch-retailer__card")]
                                            //div[contains(@class, "b-srch-card__price-new")]
                                            //span[@data-test-ref="money-base"]
                                            """)
        
    if len(n) > 0:
        if int(n[0].text) > 0:
            print(True)
    else:
        continue
        time_sleep(0.2)
print(False)


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [11]:
for _ in range(10):
    try:
        driver.find_element(By.XPATH, '//div[contains(text(), "Да. Мне есть 18")]/ancestor::button').click()
    except NoSuchElementException:
        break
    except ElementNotInteractableException:
        time_sleep(0.2)
        continue
    else:
        break

In [45]:
f1 = driver.find_elements(By.XPATH, '//li[contains(@class, "b-dsk-srch-cats-list__item")]/button')
len(f1)

2

In [44]:
su3_list = driver.find_elements(By.XPATH, '//li[contains(@class, "b-dsk-srch-cats-list__item")]/button')

if len(su3_list) > 2:
    for s_item in su3_list[1:]:
        su3 = s_item.text
        href = driver.current_url + '&segmentUuid2='+ s_item.get_attribute('value')
        print(su3, href)
else:
    su3 = np.nan
    href = driver.current_url
    print(su3, href)

Фрукты свежие https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b3355da-6311-11e6-849f-52540010b608&segmentUuid2=3b335642-6311-11e6-849f-52540010b608
Овощи https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b3355da-6311-11e6-849f-52540010b608&segmentUuid2=3b3356b2-6311-11e6-849f-52540010b608
Соленья https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b3355da-6311-11e6-849f-52540010b608&segmentUuid2=3b3357d3-6311-11e6-849f-52540010b608
Грибы свежие https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b3355da-6311-11e6-849f-52540010b608&segmentUuid2=3b335b7f-6311-11e6-849f-52540010b608
Зелень https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b3355da-6311-11e6-849f-52540010b608&segmentUuid2=3b335762-6311-11e6-849f-52540010b608
Ягоды свежие https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b3355da-6311-11e6-849f-52540010b608&segmentUuid2=3b335c1d-6311-11e6-849f-52540010b608
Зелёные салаты https://edadeal.ru/lipeck/reta

In [72]:
n = driver.find_elements(By.XPATH, """//div[contains(@class, "p-dsk-srch-retailer__content")]
                                //div[contains(@class, "b-dsk-grid__container")]
                                /a[contains(@class, "p-dsk-srch-retailer__card")]
                                
                                """)
n
# if len(n) > 0:
#     k = n[0].text
#     print(k)
# #     if int(k.text) > 0:
#         return True
    

[<selenium.webdriver.remote.webelement.WebElement (session="56843f5e568be0e9ac0389289b338551", element="8C039A430A34DBB80A60A6D8ED7078FC_element_1783")>,
 <selenium.webdriver.remote.webelement.WebElement (session="56843f5e568be0e9ac0389289b338551", element="8C039A430A34DBB80A60A6D8ED7078FC_element_1784")>,
 <selenium.webdriver.remote.webelement.WebElement (session="56843f5e568be0e9ac0389289b338551", element="8C039A430A34DBB80A60A6D8ED7078FC_element_1785")>,
 <selenium.webdriver.remote.webelement.WebElement (session="56843f5e568be0e9ac0389289b338551", element="8C039A430A34DBB80A60A6D8ED7078FC_element_1786")>,
 <selenium.webdriver.remote.webelement.WebElement (session="56843f5e568be0e9ac0389289b338551", element="8C039A430A34DBB80A60A6D8ED7078FC_element_1787")>,
 <selenium.webdriver.remote.webelement.WebElement (session="56843f5e568be0e9ac0389289b338551", element="8C039A430A34DBB80A60A6D8ED7078FC_element_1788")>,
 <selenium.webdriver.remote.webelement.WebElement (session="56843f5e568be0e9

### 3.1.2. Выполнение

In [23]:
# Получение всех адресов на страницы с товарами.
retailer_links_pages, loger_fault, q = _get_all_links_to_market(driver, retailer_links, timeout=10)
len(retailer_links_pages)

100%|██████████| 58/58 [05:57<00:00,  6.17s/it]


952

### 3.1.3. Save/Load

Ссылки на страницы с информацией

In [24]:
with open('data/retailer_links_pages.pickle', 'wb') as file:
    pickle.dump(retailer_links_pages, file)

In [8]:
with open('data/retailer_links_pages.pickle', 'rb') as file:
    retailer_links_pages = pickle.load(file)

TypeError: market_link.__new__() missing 2 required positional arguments: 'su2' and 'page_href'

## 3.2. Получение информации о товарах и ссылки на них

### 3.2.1 Функции

#### 3.2.1.1 Парсинг информации

In [26]:
def parser_sku_01(page_source: str, market: market_link, date: datetime) -> pd.DataFrame:
    """
        Извлечение данных из страницы,
        Данные возвращаются в виде pd.DataFrame
    """
    
    # Получаем хост из ссылки
    host = '/'.join(market.page_href.split('/', maxsplit=3)[:3])
    # 
    page = BS(page_source, 'lxml')
    
    # Находим карточки с продуктами
    f1 = page.find('div', {'data-test-ref':'grid'})
    f2 = f1.find('div').find_all('div', {'data-test-ref':'b-srch-card'})
    
    out_list = list()
    
    for f3 in f2:
        # название
        title = f3.find('div', class_='b-srch-card__title').get_text(strip=True)
        
        # дата до истечения
        try:
            date_to = datetime.strptime(f3.find('time').get('datetime'), '%Y-%m-%d')
        except AttributeError:
            date_to = np.nan
        
        # Цена (разбита на части)
        try:
            b = (f3
                .find('div', class_="b-srch-card__price")
                .find('div', class_="b-srch-card__price-new")
                .find('div', class_='b-money__root-wrapper')
                .find('span', {'data-test-ref':"money-base"})
                .get_text(strip=True)
                .replace('\xa0', '')
                )
        except AttributeError as ex:
            price = np.nan
            short_logger.critical(f"no price, {ex}: {market.title} :: {market.page_href}")
            long_logger.critical(f"no price, {ex}: {market.title} :: {market.page_href}", exc_info=True)
        
        else:
            try:
                s = (f3
                    .find('div', class_="b-srch-card__price")
                    .find('div', class_="b-srch-card__price-new")
                    .find('div', class_='b-money__root-wrapper')
                    .find('span', {'data-test-ref':"money-subunit"})
                    .get_text(strip=True)
                    .replace('\xa0', ''))
            except AttributeError:
                s = '0'
                
            price = float(f'{b}.{s}')
        
        # -------------------------------------------------------------------------------------------------
        # Предыдущая цена (разбита на части)
        try:
            b_last = (f3
                      .find('div', class_="b-srch-card__price")
                      .find('div', class_="b-srch-card__price-old")
                      .find('div', class_='b-money__root-wrapper')
                      .find('span', {'data-test-ref':"money-base"})
                      .get_text(strip=True)
                      .replace('\xa0', '')
                      )
        except AttributeError as ex:
            price_last = np.nan
        
        else:
            try:
                s_last = (f3
                          .find('div', class_="b-srch-card__price")
                          .find('div', class_="b-srch-card__price-old")
                          .find('div', class_='b-money__root-wrapper')
                          .find('span', {'data-test-ref':"money-subunit"})
                          .get_text(strip=True)
                          .replace('\xa0', '')
                          )
            except AttributeError:
                s_last = '0'
                
            price_last = float(f'{b_last}.{s_last}')

        # -------------------------------------------------------------------------------------------------
        # тип валюты
        try:
            unit = f3.find('div', class_='b-money__root-wrapper').find('span', {'data-test-ref':"currency"}).get_text(strip=True)
        except AttributeError:
            unit = np.nan

        # -------------------------------------------------------------------------------------------------
        # скидка
        try:
            sale = f3.find('div', class_='b-srch-card__discount-label').get_text(strip=True)
        except AttributeError:
            sale = np.nan
        
        # -------------------------------------------------------------------------------------------------
        # Условие
        try:
            condition = (f3
                         .find('div', class_="b-srch-card__content-container")
                         .find('div', class_="b-tpl-offer-label__text")
                         .get_text(strip=True)
                         .replace('\xa0', '')
                         )
        except AttributeError as ex:
                condition = np.nan

        # -------------------------------------------------------------------------------------------------
        # вес
        try:
            f4 = (f3
                .find('div', class_='b-srch-card__info')
                .find('span')
                .get_text(strip=True)
                .replace('\xa0', ' ')
                .split('•')
                )
            vol = f4[0].strip()
        
        # руб/кг. руб/шт.
            vol_vs = f4[1].strip()
        except AttributeError:
            vol = np.nan
            vol_vs = np.nan
                
        # -------------------------------------------------------------------------------------------------
        # ссылка
        try:
            href = host + f3.find('a', class_="b-srch-card__card-wrapper").get('href')
        except AttributeError:
            href = np.nan
        
                
        out_list.append((date, market.sity, market.title, title, date_to, condition ,price, price_last, unit, sale, vol, vol_vs, href))


    return pd.DataFrame(out_list, columns=['date', 'sity', 'market', 'title', 'date_to', 'condition', 'price', 'price_last', 'unit', 'sale', 'vol', 'vol_vs', 'href'])

#### 3.2.1.2 Получение страницы

In [27]:
def _get_page(driver: webdriver.Chrome, item: market_link, timeout: int=30) -> webdriver.Chrome.page_source:
    """
        сохраняем страницы
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1
        )
    
    # Вызов страницы
    try:
        driver.get(item.page_href)
    
    except TimeoutException as ex:
        short_logger.warning(f"don't get response: {item.title} :: {item.page_href}")
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    except Exception as ex:
        short_logger.critical(f"other exception, {ex}: {item.title} :: {item.page_href}")
        long_logger.critical(f"other exception, {ex}: {item.title} :: {item.page_href}", exc_info=True)
    
    # Проверка сайта на 18 лет
    # Мы ранее собирали информацию, и приблизительно занем есть ли в данном магазине такая проверка или нет.
    if item.alcohol:
        try:
            # Ожидаем появляение кнопки, если она не повится, то ничего не делаем.
            button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
            button.click()
        
        except TimeoutException:
            short_logger.info(f'alcochol timeout: {item.title} :: {item.page_href}')
        except ElementNotInteractableException:
            short_logger.warning(f'alcochol not button: {item.title} :: {item.page_href}')
        except Exception as ex:
            short_logger.error(f'alcochol other: {ex} :: {item.title} :: {item.page_href}')
            long_logger.error(f'alcochol other: {ex} :: {item.title} :: {item.page_href}', exc_info=True)
    
    # Ручной тайм аут на выполнение всего процесса
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout*2:
        
        # Задержка действий до загрузки отслеживаемых елементов.
        # Варант провекрки загрузки страницы, проверка что текст не пустой
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@data-test-ref='grid']"))
        except TimeoutException:
            try:
                driver.find_element(by=By.XPATH, value="//div[@class='b-dsk-stub__description']")
            except TimeoutException:
                short_logger.warning(f"not table, timeout: {item.title} :: {item.page_href}")
                raise MyTimeoutErrore()
            else:
                short_logger.info(f"not page, timeout: {item.title} :: {item.page_href}")
                page = None
                break
            
        # Проверка наличия названия продукта в ячейке
        page = BS(driver.page_source, 'lxml')
        
        try:
            # Дожидаемся когда загрузится информация о первом товаре на странице
            f1 = page.find('div', {'data-test-ref':'grid'})
            f2 = f1.find('div').find_all('div', {'data-test-ref':'b-srch-card'})
            f2[0].find('div', class_='b-srch-card__title').get_text(strip=True)
        except IndexError as ex:
            continue
        except Exception as ex:
            short_logger.critical(f"checking the product name, other exception, {ex}: {item.title} :: {item.page_href}")
            long_logger.critical(f"checking the product name, other exception, {ex}: {item.title} :: {item.page_href}", exc_info=True)

        break
    
    else:
        short_logger.warning(f"not table, timeout ALL: {item.title} :: {item.page_href}")
        raise MyTimeoutErrore()
    
    return driver.page_source

#### 3.2.1.3. Получение информации с сайта.

In [28]:
def get_pages(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=20, dead_line: int=20) -> tuple[pd.DataFrame, list[market_link]]:
    """
        Функция получает страницы с информацией об товарах,
        извлекает информацию,
        отдает датафрейм с информацией и необработанными ссылками.
    """
    table = list()
    
    with tqdm(total=len(list_url)) as pbar:     # Прогресс бар
        
        q = deque(list_url)                     # Очередь
        out_list = list()
        
        while len(q) > 0:
            if dead_line < 0:                   # Ограничение количества попыток получить данные.
                return (out_list, q)
            
            item = q.popleft()
            if item.discounts:
                try:
                    page = _get_page(driver, item, timeout)
                except MyTimeoutErrore:
                    q.append(item)
                    dead_line -= 1
                    continue
                else:
                    table.append(parser_sku_01(page, item, datetime.now()))
                
            pbar.update(1)
        df = pd.concat(table, ignore_index=True)
    return (df, list(q))                        # Возвращаем отработаный лист и очередь(пустую или нет)

### 3.2.2. Выполнение

In [30]:
pages_df, other = get_pages(driver, retailer_links_pages)

100%|██████████| 952/952 [37:30<00:00,  2.36s/it]  


In [37]:
_save_df_to_zip(pages_df)

#### 3.2.2.1. Сохранение всех страниц, для того что бы отточить работу парсера.

In [19]:
# pages, other = _get_pages(driver, retailer_links_pages)

100%|██████████| 914/914 [35:32<00:00,  2.33s/it]  


In [20]:
# Сохранение страниц  в Zip файл.
with ZipFile('data/pages.zip', 'w', compression=ZIP_DEFLATED, compresslevel=1) as zip_file:
    for index, page in enumerate(pages):
        if page:
            zip_file.writestr(f'{index:05d}.html', page.encode())

In [6]:
# Поллучение страниц из Zip файла.
pages = list()
with ZipFile('data/pages.zip', 'r') as zf:
    for item in zf.filelist:
        with zf.open(item.filename) as file:
            pages.append(file.read())

In [7]:
url = retailer_links_pages[0].page_href


In [23]:


dataframe_list = list()

for page in pages[:200]:
    dataframe_list.append(parser_sku_01(page, retailer_links_pages[0], datetime.strptime("2023-08-28", '%Y-%m-%d')))
out_df = pd.concat(dataframe_list, ignore_index=True)

(24, 13)